# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File '00-start-eva-server.ipynb' already there; not retrieving.



Note: you may need to restart the kernel to use updated packages.


nohup eva_server > eva.log 2>&1 &


Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip

# unzip
!unzip -n bddtest.zip

File 'bddtest.zip' already there; not retrieving.



Archive:  bddtest.zip


## Load 2 BDD videos

In [4]:
cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.10034518875181675
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.06872199289500713


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.05400330200791359


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a2e3ca5c856cde.csv                     3401
@query_time: 0.46026498451828957


@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a395fed60c0b47.csv                     4355
@query_time: 0.5280115809291601


## Run a Query over the Video dataset


In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 3;""")
response = cursor.fetch_all()
response.as_df()

,bddtest_1.id,fastrcnnobjectdetector.labels,fastrcnnobjectdetector.bboxes,fastrcnnobjectdetector.scores
0,0,"[stop sign, car, stop sign, car, car, car, car...","[[343.09747, 195.47418, 405.7211, 258.95984], ...","[0.99814737, 0.9964282, 0.9947148, 0.9944522, ..."
1,1,"[stop sign, car, car, stop sign, car, car, car...","[[334.63928, 191.12814, 398.79654, 254.96082],...","[0.9978867, 0.99533, 0.9949798, 0.9941269, 0.9..."
2,2,"[stop sign, car, car, stop sign, car, car, car...","[[327.6995, 189.36577, 392.4098, 253.57695], [...","[0.99903834, 0.9947266, 0.9944389, 0.9922162, ..."


## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
response.as_df()

,bddtestmeta._row_id,bddtestmeta.id,bddtestmeta.frame_id,bddtestmeta.video_id,bddtestmeta.dataset_name,bddtestmeta.label,bddtestmeta.bbox,bddtestmeta.object_id
0,1,8185,0,4,bddtest,car,"[491.7783, 320.9238, 717.0901, 512.1478]",512
1,2,8186,0,4,bddtest,car,"[928.2679, 298.47574, 1279.5381, 571.17786]",513
2,3,8187,0,4,bddtest,car,"[816.0277, 347.52887, 1029.6998, 474.7344]",514
3,4,8188,0,4,bddtest,car,"[811.87067, 325.91223, 948.2217, 408.2217]",515
4,5,8189,0,4,bddtest,car,"[673.0254, 325.08084, 751.17786, 367.48267]",516
...,...,...,...,...,...,...,...,...
3396,3397,11581,1197,4,bddtest,car,"[554.96533, 335.88916, 588.2217, 378.291]",643
3397,3398,11582,1197,4,bddtest,car,"[693.8106, 350.0231, 734.5497, 395.75058]",644
3398,3399,11583,1197,4,bddtest,car,"[683.83374, 345.03464, 733.71826, 389.93073]",642
3399,3400,11584,1197,4,bddtest,car,"[676.351, 346.69745, 699.6305, 384.11084]",641
